In [2]:
import pandas as pd
import os
import requests
import json
import base64
import time
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [3]:
current = pd.read_clipboard()

In [4]:
for i in range(current.shape[0]):
    for j in range(current.shape[1]):
        a = current.iat[i, j].split('track-id=')[-1]
        if a == current.iat[i, j]:
            b = current.iat[i, j].split('track/')[-1]
            b = b.split('?')[0]
            if b == current.iat[i, j]:
                continue
            else:
                current.iat[i, j] = b
        else:
            current.iat[i, j] = a
            
current['day'] = range(100, 100-len(current), -1)
result = pd.DataFrame(columns=['track_id', 'Listers', 'day'])
for i in range(current.shape[0]):
    for j in range(current.shape[1]-1):
        if current.iat[i, j] == 'track_id':
            continue
        else:
            result = result.append({'track_id': current.iat[i, j], 'Listers': current.columns[j], 'day': current.iat[i, -1]}, ignore_index=True)

/var/folders/b9/1w_dn_kj5zgffq25ztcmwj5m0000gn/T/ipykernel_12002/3546298466.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'track_id': current.iat[i, j], 'Listers': current.columns[j], 'day': current.iat[i, -1]}, ignore_index=True)
/var/folders/b9/1w_dn_kj5zgffq25ztcmwj5m0000gn/T/ipykernel_12002/3546298466.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'track_id': current.iat[i, j], 'Listers': current.columns[j], 'day': current.iat[i, -1]}, ignore_index=True)
/var/folders/b9/1w_dn_kj5zgffq25ztcmwj5m0000gn/T/ipykernel_12002/3546298466.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'track_id': current.iat[i, j], 'Listers': current.column

In [5]:
def get_token():
    url = 'https://accounts.spotify.com/api/token'
    auth_str = client_id + ':' + client_secret
    auth_bytes = auth_str.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')
    headers = {
        'Authorization': 'Basic ' + auth_base64,
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {
        'grant_type': 'client_credentials'
    }
    response = requests.post(url, headers=headers, data=data)
    return json.loads(response.content)['access_token']

def get_auth_header():
    token = get_token()
    return {'Authorization': 'Bearer ' + token}

In [6]:
# write a fuction to go through each row  100 at a time in the dataframe and get track info and return a new dataframe with the track info
def batch_get_track_info(df):
    track_ids = df['track_id'].tolist()
    df['Featured Artist'] = ''
    url = 'https://api.spotify.com/v1/tracks'
    l = len(track_ids)
    i = 0
    n = 50
    while i < l:
        if i + n < l:
            batch = track_ids[i:i + n]
        else:
            batch = track_ids[i:]
        params = {
            'ids': ','.join(batch)
        }
        headers = get_auth_header()
        response = requests.get(url, headers=headers, params=params)
        data = json.loads(response.content)

        for j, track in enumerate(data['tracks']):
            df.at[i + j, 'Duration'] = track['duration_ms']
            df.at[i + j, 'Release Date'] = track['album']['release_date']
            df.at[i + j, 'Popularity'] = track['popularity']
            df.at[i + j, 'Album'] = track['album']['name']
            df.at[i + j, 'Artist'] = track['artists'][0]['name']
            df.at[i + j, 'Song'] = track['name']
            if len(track['artists']) > 1:
                df.at[i + j, 'Featured Artist'] = track['artists'][1]['name']
        i += n
    return df


In [7]:
output = batch_get_track_info(result)
output['track_id_stable'] = output['Song']+ '' + output['Artist']
final = output[['track_id_stable', 'Featured Artist', 'Duration', 'Release Date', 'Popularity', 'Album', 'Artist', 'Song']]
# remove duplicates
final = final.drop_duplicates(subset='track_id_stable')
final['Zach'] = ''
final['Bryce'] = ''
final['Maggie'] = ''
final['Jamie'] = ''
for idx, row in final.iterrows():
    for i in range(len(output)):
        if row['track_id_stable'] == output.loc[i, 'track_id_stable']:
            if output.loc[i, 'Listers'] == 'Zach':
                final.at[idx, 'Zach'] = output.loc[i, 'day']
            elif output.loc[i, 'Listers'] == 'Bryce':
                final.at[idx, 'Bryce'] = output.loc[i, 'day']
            elif output.loc[i, 'Listers'] == 'Maggie':
                final.at[idx, 'Maggie'] = output.loc[i, 'day']
            elif output.loc[i, 'Listers'] == 'Jamie':
                final.at[idx, 'Jamie'] = output.loc[i, 'day']
final = final[['track_id_stable', 'Zach', 'Bryce', 'Maggie', 'Jamie', 'Featured Artist', 'Duration', 'Release Date', 'Popularity', 'Album', 'Artist', 'Song']]
final = final.reset_index(drop=True)


In [10]:
import datetime

def search_song(title, artist, feature):
    
    url = 'https://api.spotify.com/v1/search'
    headers = get_auth_header()
    params = {
        'q': 'track:' + title + ' artist:' + artist,
        'type': 'track'
    }
    response = requests.get(url, headers=headers, params=params)
    data = json.loads(response.content)

    album = ""
    release_date = datetime.datetime(2030, 1, 1)
    release_date_album = datetime.datetime(2030, 1, 1)
    popularity = -1
    for i in range(len(data['tracks']['items'])):
        featured = False
        if feature == "":
            featured = True
        elif len(data['tracks']['items'][i]['artists']) > 1 and feature != "":
            featured = data['tracks']['items'][i]['artists'][1]['name'] == feature
        if featured and data['tracks']['items'][i]['name'] == title and data['tracks']['items'][i]['artists'][0]['name'] == artist:
            if data['tracks']['items'][i]['album']['release_date_precision'] == 'day':
                year = int(data['tracks']['items'][i]['album']['release_date'][:4])
                month = int(data['tracks']['items'][i]['album']['release_date'][5:7])
                day = int(data['tracks']['items'][i]['album']['release_date'][8:10])
            elif data['tracks']['items'][i]['album']['release_date_precision'] == 'month':
                year = int(data['tracks']['items'][i]['album']['release_date'][:4])
                month = int(data['tracks']['items'][i]['album']['release_date'][5:7])
                day = 1
            elif data['tracks']['items'][i]['album']['release_date_precision'] == 'year':
                year = int(data['tracks']['items'][i]['album']['release_date'][:4])
                month = 1
                day = 1
                
            if datetime.datetime(year, month, day) < release_date:
                release_date = datetime.datetime(year, month, day)
            if data['tracks']['items'][i]['popularity'] > popularity:
                popularity = data['tracks']['items'][i]['popularity']    
                duration = data['tracks']['items'][i]['duration_ms']
                track_id = data['tracks']['items'][i]['id']
            if data['tracks']['items'][i]['album']['album_type'] == 'album' and datetime.datetime(year, month, day) < release_date_album:
                album = data['tracks']['items'][i]['album']['name']
                release_date_album = datetime.datetime(year, month, day)
            
    return(duration, release_date, popularity, album, title, artist, track_id)
    
    

In [13]:
from tqdm import tqdm
errors_in_a_row = 0
last_checked = 356
for i in tqdm(range(final.shape[0])):
    if i > last_checked:
        try:
            duration, release_date, popularity, album, song_title, artist, track_id = search_song(final.loc[i, 'Song'], final.loc[i, 'Artist'], final.loc[i, 'Featured Artist'])
            final.at[i, 'Duration'] = duration
            final.at[i, 'Release Date'] = release_date
            final.at[i, 'Popularity'] = popularity
            final.at[i, 'Album'] = album
            errors_in_a_row = 0
        except:
            errors_in_a_row += 1
            print("Error at row " + str(i) + ": " + final.loc[i, 'Song'] + " by " + final.loc[i, 'Artist'])
            if errors_in_a_row >= 5:
                print("Too many errors in a row, stopping.")
                break
            continue
        time.sleep(10)

100%|██████████| 357/357 [1:01:20<00:00, 10.31s/it]


In [14]:
final.to_csv('2024_spotify_data.csv', index=False)

In [17]:
final

,track_id_stable,Zach,Bryce,Maggie,Jamie,Featured Artist,Duration,Release Date,Popularity,Album,Artist,Song
0,Lookin' for LoveOld Dominion,100,,,,,210120.0,2024-09-06,37.0,Odies But Goodies,Old Dominion,Lookin' for Love
1,Nonsense (with Coi Leray) - RemixSabrina Carpe...,,100,,,Coi Leray,175648.0,2023-03-23,38.0,,Sabrina Carpenter,Nonsense (with Coi Leray) - Remix
2,You've Got The LoveFlorence + The Machine,,,100,,,168666.0,2009-01-01,60.0,Lungs (Deluxe Edition),Florence + The Machine,You've Got The Love
3,Soak Up The SunSheryl Crow,,,,100,,292306.0,2002-04-08,57.0,"C'Mon, C'Mon",Sheryl Crow,Soak Up The Sun
4,Edge of the EarthThe Beaches,99,,,,,160244.0,2023-05-05,57.0,Blame My Ex,The Beaches,Edge of the Earth
...,...,...,...,...,...,...,...,...,...,...,...,...
352,There’s Your Trouble - Spotify Singles (Live f...,,7,,,,191949.0,2024-07-24,45.0,,mgk,There’s Your Trouble - Spotify Singles (Live f...
353,Nights On BroadwayBee Gees,,,7,,,271373.0,1975-06-01,51.0,Main Course,Bee Gees,Nights On Broadway
354,AlrightCouch,,,,7,,250105.0,2023-10-06,35.0,,Couch,Alright
355,Game NightDJ Heer,,6,,,A.S.M,231682.0,2024-04-18,2.0,,DJ Heer,Game Night


In [16]:
final['Release Date'] = final['Release Date'].astype(str)
final['Release Date'] = final['Release Date'].str.replace(' 00:00:00', '')
final.to_clipboard(index=False, header=False)